In [1]:
import cv2
import numpy as np
import os
import sys
import subprocess
import shutil

In [2]:
def is_valid_image_file(filename):
    """
    Check if the file is a valid image format for processing (excluding .heic).
    """
    valid_extensions = ['.jpg', '.jpeg', '.png']
    return any(filename.lower().endswith(ext) for ext in valid_extensions)

In [3]:
def detect_faces(input_dir, output_dir):
    # Load the pre-trained Haar Cascade model for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate over subdirectories in the main directory
    for subdir in os.listdir(input_dir):
        subdir_path = os.path.join(input_dir, subdir)
        output_subdir = os.path.join(output_dir, subdir)
        os.makedirs(output_subdir, exist_ok=True)

        print("Processing directory:", subdir_path)

        # Iterate over files in the current subdirectory
        for filename in os.listdir(subdir_path):
            image_path = os.path.join(subdir_path, filename)

            try:
                image = cv2.imread(image_path)

                if image is not None:
                    # Convert the image to grayscale for face detection
                    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    
                    for i, (x, y, w, h) in enumerate(faces):
                        face_region = image[y:y+h, x:x+w]
                        resized_face = cv2.resize(face_region, (224, 224))
                        output_filename = f"{os.path.splitext(filename)[0]}_face{i}{os.path.splitext(filename)[1]}"
                        output_path = os.path.join(output_subdir, output_filename)

                        cv2.imwrite(output_path, resized_face)
                        print(f"Saved face: {output_path}")
                else:
                    print(f"Error loading image: {image_path}")
            except Exception as e:
                print(f"An error occurred while processing {image_path}: {str(e)}")

    print("Face extraction completed.")

In [4]:
def enhance_contrast(img, intensity):
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    l = np.clip(cv2.add(l, intensity), 0, 255)
    merged_lab = cv2.merge((l, a, b))
    gray_img = cv2.cvtColor(cv2.cvtColor(merged_lab, cv2.COLOR_LAB2BGR), cv2.COLOR_BGR2GRAY)
    return gray_img

def enhance_brightness(img, intensity):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_img)
    v = np.clip(cv2.add(v, intensity), 0, 255)
    merged_hsv = cv2.merge((h, s, v))
    gray_img = cv2.cvtColor(cv2.cvtColor(merged_hsv, cv2.COLOR_HSV2BGR), cv2.COLOR_BGR2GRAY)
    return gray_img

def process_images(source, destination):
    """
    Process images by applying various transformations and saving them.
    """
    if not os.path.exists(destination):
        os.makedirs(destination)

    for root_dir, _, filenames in os.walk(source):
        dest_path = os.path.join(destination, os.path.relpath(root_dir, source))
        os.makedirs(dest_path, exist_ok=True)

        for file in filenames:
            if not is_valid_image_file(file):
                continue
    if not os.path.exists(destination):
        os.makedirs(destination)

    for root_dir, _, filenames in os.walk(source):
        dest_path = os.path.join(destination, os.path.relpath(root_dir, source))
        os.makedirs(dest_path, exist_ok=True)

        for file in filenames:
            if file.lower().endswith((".jpg", ".png")):
                img_path = os.path.join(root_dir, file)
                img = cv2.imread(img_path)

                if img is None or img.shape[-1] != 3:
                    continue

                denoised_img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

                gray_img = cv2.cvtColor(denoised_img, cv2.COLOR_BGR2GRAY)
                blur_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
                bright_img = enhance_brightness(denoised_img, 50)
                dark_img = enhance_brightness(denoised_img, -50)
                high_contrast_img = enhance_contrast(denoised_img, 50)
                low_contrast_img = enhance_contrast(denoised_img, -50)
                mirror_img = cv2.flip(gray_img, 1)

                # Saving the processed images
                cv2.imwrite(os.path.join(dest_path, f"{file.split('.')[0]}_gray.jpg"), gray_img)
                cv2.imwrite(os.path.join(dest_path, f"{file.split('.')[0]}_blur.jpg"), blur_img)
                cv2.imwrite(os.path.join(dest_path, f"{file.split('.')[0]}_bright.jpg"), bright_img)
                cv2.imwrite(os.path.join(dest_path, f"{file.split('.')[0]}_dark.jpg"), dark_img)
                cv2.imwrite(os.path.join(dest_path, f"{file.split('.')[0]}_high_contrast.jpg"), high_contrast_img)
                cv2.imwrite(os.path.join(dest_path, f"{file.split('.')[0]}_low_contrast.jpg"), low_contrast_img)
                cv2.imwrite(os.path.join(dest_path, f"{file.split('.')[0]}_mirror.jpg"), mirror_img)

In [6]:
def main(input_dir, output_dir):
    # Load the pre-trained Haar Cascade model for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')

    face_out_dir = os.path.join(output_dir, "face/")
    combined_out_dir = os.path.join(output_dir, "combined/")
    aug_out_dir = os.path.join(output_dir, "augment/")
    final_out_dir = os.path.join(output_dir, "train/")

    # Detect faces and save in face_out_dir
    detect_faces(input_dir, face_out_dir)

    # Process the detected faces for data augmentation and save in aug_out_dir
    process_images(face_out_dir, aug_out_dir)

    # Optionally, combine original and augmented data
    # This step is optional and depends on whether you want to combine the original processed images with the augmented ones
    # If so, uncomment the next two lines
    # for filename in os.listdir(aug_out_dir):
    #     shutil.copy(os.path.join(aug_out_dir, filename), combined_out_dir)

    # Move augmented data to final directory
    if os.path.exists(aug_out_dir) and os.listdir(aug_out_dir):
        shutil.move(aug_out_dir, final_out_dir)
        print("Moved augmented data to final directory.")
    else:
        print(f"Augmented directory {aug_out_dir} does not exist or is empty.")
    print("Final output directory:", final_out_dir)

In [7]:
input_directory = "/content/drive/MyDrive/COEN 240/Release-2"
output_directory = "/content/drive/MyDrive/COEN 240"

# Call the main function 
main(input_directory, output_directory)

Processing directory: COEN240/Release-2/Kimsong
Saved face: COEN240/face/Kimsong/Capture7_face0.PNG
Saved face: COEN240/face/Kimsong/Capture6_face0.PNG
Saved face: COEN240/face/Kimsong/Capture4_face0.PNG
Saved face: COEN240/face/Kimsong/Capture1_face0.PNG
Saved face: COEN240/face/Kimsong/Capture2_face0.PNG
Saved face: COEN240/face/Kimsong/Capture8_face0.PNG
Saved face: COEN240/face/Kimsong/Capture_face0.PNG
Saved face: COEN240/face/Kimsong/Screenshot 2023-09-26 at 12.40.58 PM_face0.png
Error loading image: COEN240/Release-2/Kimsong/IMG_8546.HEIC
Processing directory: COEN240/Release-2/Anirudh
Saved face: COEN240/face/Anirudh/0926-ClassPhotos_face0.png
Error loading image: COEN240/Release-2/Anirudh/IMG_8564.HEIC
Saved face: COEN240/face/Anirudh/Screenshot 2023-10-21 at 9.57.15 PM_face0.png
Saved face: COEN240/face/Anirudh/9-28_face0.png
Saved face: COEN240/face/Anirudh/0921-ClassPhotos _face0.png
Saved face: COEN240/face/Anirudh/0921-ClassPhotos _face1.png
Saved face: COEN240/face/Aniru